In [9]:
import pandas as pd
import os
import re
import unicodedata
from rapidfuzz import fuzz, process as fuzzprocess
import numpy as np
import os
import unicodedata
import pandas as pd

In [10]:
_re_non_alnum = re.compile(r"[^0-9a-z]+")
_re_multi_unders = re.compile(r"_+")

def normalizar_texto(texto: str) -> str:
    """Normaliza texto : sin acentos, minúsculas, underscores limpios."""
    if not isinstance(texto, str):
        return texto

    texto = unicodedata.normalize("NFKD", texto)
    texto = "".join(c for c in texto if not unicodedata.combining(c))
    texto = texto.lower()

    texto = _re_non_alnum.sub("_", texto)
    texto = _re_multi_unders.sub("_", texto)
    return texto.strip("_")

In [11]:
def normalize_sheet_name(name):
    """Normaliza y singulariza para evitar 'vehiculos' / 'vehiculo'."""
    norm = normalizar_texto(name)
    # singularización simple
    if norm.endswith("es") and len(norm) > 3:
        norm = norm[:-2]
    elif norm.endswith("s") and len(norm) > 2:
        norm = norm[:-1]
    return norm

In [12]:
# 

root_excels = r"C:\\Users\\hiros\\Desktop\\CMS-PRUEBA-TECNICA\\descargas"

MESES = {
    "enero": 1, "febrero": 2, "marzo": 3, "abril": 4,
    "mayo": 5, "junio": 6, "julio": 7, "agosto": 8,
    "septiembre": 9, "octubre": 10, "noviembre": 11, "diciembre": 12
}
años = ['2025','2024','2023','2022','2021','2020','2019','2018','2017','2016','2015','2014','2013','2012','2011','2010']


def extraer_mes_nomb_excel(nombre_archivo):
    # quitar extensión
    nombre = os.path.splitext(nombre_archivo)[0].lower()

    # separar por espacios, guiones, underscores, etc.
    partes = re.split(r"[ _\-]+", nombre)

    # buscar coincidencia exacta
    for parte in partes:
        if parte in MESES:
            return parte  # retorna el nombre del mes encontrado

    # búsqueda más flexible: " - diciembre 2012"
    for mes in MESES:
        if mes in nombre:
            return mes

    return None


def extraer_año(nombre_archivo):
    
    # quitar extensión
    nombre = os.path.splitext(nombre_archivo)[0].lower()

    # separar por espacios, guiones, underscores, etc.
    partes = re.split(r"[ _\-]+", nombre)

    # buscar coincidencia exacta
    for parte in partes:
        if parte in años:
            return parte  # retorna el año encontrado

    # búsqueda más flexible: " - diciembre 2012"
    for año in años:
        if año in nombre:
            return año

    return None

In [13]:
def es_hoja_vehiculo(sheet_name: str) -> bool:
    norm = normalize_sheet_name(sheet_name)  # e.g. "Servicios" → "servicios"
    tokens = norm.split("_")

    # 1) Token empieza con "veh" → MUY seguro
    for t in tokens:
        if t.startswith("veh"):
            return True

    # 2) Excluir explícitamente cosas tipo "servicio" / "servicios"
    if any(t.startswith(("serv", "servi", "servic")) for t in tokens):
        return False

    # 3) Fuzzy matching seguro contra "vehiculo"
    score = fuzz.partial_ratio(norm, "vehiculo")
    if score >= 75:
        return True

    # 4) Fuzzy por token con longitud mínima (evita match con "ser")
    for t in tokens:
        if len(t) >= 3:
            if fuzz.partial_ratio(t, "veh") >= 90:
                return True

    return False


In [14]:
def leer_excels_vehiculos(root_path):
    todos = []

    for year_folder in os.listdir(root_path):
        ruta_año = os.path.join(root_path, year_folder)
        if not os.path.isdir(ruta_año):
            continue

        for archivo in os.listdir(ruta_año):
            if not archivo.lower().endswith((".xlsx", ".xls", ".xlsm")):
                continue

            ruta_file = os.path.join(ruta_año, archivo)

            año_det = extraer_año(archivo)
            mes_det = extraer_mes_nomb_excel(archivo)

            try:
                excel = pd.ExcelFile(ruta_file)
            except Exception as e:
                print(f"❌ Error abriendo {archivo}: {e}")
                continue

            # Flag para saber si encontramos una hoja válida
            encontro_hoja = False

            for hoja in excel.sheet_names:
                if es_hoja_vehiculo(hoja):
                    print(f"✔ Leyendo {archivo} | hoja automática: {hoja}")
                    try:
                        df = pd.read_excel(ruta_file, sheet_name=hoja)
                        df["año_archivo"] = año_det
                        df["mes_archivo"] = mes_det
                        df["archivo"] = archivo
                        df["hoja"] = hoja
                        todos.append(df)
                        encontro_hoja = True
                    except Exception as e:
                        print(f"❌ Error leyendo hoja {hoja} en {archivo}: {e}")
                    continue

            # -----------------------------------------
            # Fallback interactivo si NO encontró nada
            # -----------------------------------------
            if not encontro_hoja:
                print(f"\n⚠ No encontré ninguna hoja tipo 'vehiculo' en el archivo:")
                print(f"   → {archivo}")
                print("   Hojas disponibles:")

                for i, h in enumerate(excel.sheet_names):
                    print(f"   [{i}] {h}")

                try:
                    idx = int(input(f"👉 Ingresa el número de la hoja que quieres usar (o -1 para saltar) opciones{list(range(len(excel.sheet_names)))}: "))
                except:
                    print("Entrada inválida. Saltando archivo.")
                    continue

                if idx == -1:
                    print("⏭ Saltando archivo.")
                    continue
                if idx < 0 or idx >= len(excel.sheet_names):
                    print("❌ Índice fuera de rango. Saltando archivo.")
                    continue

                hoja_manual = excel.sheet_names[idx]
                print(f"✔ Leyendo manualmente: {archivo} | hoja: {hoja_manual}")

                try:
                    df = pd.read_excel(ruta_file, sheet_name=hoja_manual)
                    df["año_archivo"] = año_det
                    df["mes_archivo"] = mes_det
                    df["archivo"] = archivo
                    df["hoja"] = hoja_manual
                    todos.append(df)
                except Exception as e:
                    print(f"❌ Error leyendo hoja manual {hoja_manual} en {archivo}: {e}")
                    continue

    if todos:
        return pd.concat(todos, ignore_index=True)
    return pd.DataFrame()


In [15]:
df_vehiculos = leer_excels_vehiculos(root_excels)
df_vehiculos 


✔ Leyendo Base de Datos de Transporte Público - Abril 2021.xlsx | hoja automática: Vehículos
✔ Leyendo Base de Datos de Transporte Público - Agosto 2021.xlsx | hoja automática: Vehículo
✔ Leyendo Base de Datos de Transporte Público - Diciembre 2021.xlsx | hoja automática: Vehículo
✔ Leyendo Base de Datos de Transporte Público - Enero 2021.xlsx | hoja automática: Vehiculos
✔ Leyendo Base de Datos de Transporte Público - Febrero 2021.xlsx | hoja automática: Vehículo
✔ Leyendo Base de Datos de Transporte Público - Julio 2021.xlsx | hoja automática: Vehículos
✔ Leyendo Base de Datos de Transporte Público - Junio 2021.xlsx | hoja automática: Vehiculos
✔ Leyendo Base de Datos de Transporte Público - Marzo 2021.xlsx | hoja automática: Vehiculos vig
✔ Leyendo Base de Datos de Transporte Público - Mayo 2021.xlsx | hoja automática: Vehiculos
✔ Leyendo Base de Datos de Transporte Público - Noviembre 2021.xlsx | hoja automática: Vehículo
✔ Leyendo Base de Datos de Transporte Público - Octubre 2021

,REGION,FOLIO,TIPO_SERVICIO,PPU,FECHA_INGRESO_RNT,MARCA,MODELO,ANO_FABRICACION,CAPACIDAD,año_archivo,...,TIPO_SERVCIO,TIPO_COMBUSTIBLE,FECHA_INGRESO_RNT.1,TIPO_COMBUSTIBLE,COMBUSTIBLE,FECHA_INGRESO_RNTT,TIPO SERVICIO,LINEA,AÑO FABRICACION,AÑOFABRICACION
0,1.0,10,TAXI COLECTIVO RURAL,RW2882,2007-06-26,HYUNDAI,ACCENT GL 1.5,2007.0,4.0,2021,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.0,12,TAXI COLECTIVO RURAL,BCRH67,2019-10-07,HYUNDAI,ACCENT 1.6,2015.0,NaN,2021,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1.0,18,TAXI COLECTIVO RURAL,DWCY37,2012-05-15,HYUNDAI,ACCENT RB GL 1.6,2012.0,4.0,2021,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1.0,19,TAXI COLECTIVO RURAL,DGFX88,2011-08-30,HYUNDAI,ACCENT RB GL 1.6,2011.0,4.0,2021,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1.0,24,TAXI COLECTIVO RURAL,FPCP93,2013-05-03,HYUNDAI,ACCENT RB GL 1.6,2013.0,4.0,2021,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10480787,16.0,500700,NaN,DKWJ44,2013-06-03-15.58.03.000000,SCANIA,K380B,NaN,44.0,2019,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10480788,16.0,500936,NaN,BXDK27,2015-11-10-17.26.01.000000,MERCEDES BENZ,0500 RS,NaN,NaN,2019,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10480789,16.0,500825,NaN,KXBT57,2018-11-09-00.00.00.000000,MERCEDES BENZ,OF 1724,NaN,46.0,2019,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10480790,16.0,400080,NaN,WK7479,2007-08-10-12.00.00.000000,MERCEDES BENZ,LO 914 42,NaN,NaN,2019,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
import datetime

def detect_and_convert_datetime(df, sample_size=100):
	"""Detect object columns that contain datetimes and convert them to datetime64[ns]."""
	df = df.copy()
	obj_cols = df.select_dtypes(include=["object"]).columns
	date_cols = []
	for col in obj_cols:
		nonnull = df[col].dropna()
		if nonnull.empty:
			continue
		sample = nonnull.head(sample_size)
		# If any sample item is a Timestamp / datetime, treat column as datetime
		if sample.map(lambda x: isinstance(x, (pd.Timestamp, datetime.datetime))).any():
			date_cols.append(col)
			continue
		# Otherwise try parsing the sample; if any parses, treat as datetime
		parsed = pd.to_datetime(sample, errors="coerce", infer_datetime_format=True)
		if parsed.notna().any():
			date_cols.append(col)

	# Convert detected columns
	for col in date_cols:
		df[col] = pd.to_datetime(df[col], errors="coerce", infer_datetime_format=True)

	return df, date_cols

df_vehiculos, converted_cols = detect_and_convert_datetime(df_vehiculos)
print("Converted to datetime:", converted_cols)

# Ensure object columns are converted to pandas string dtype to avoid pyarrow ArrowTypeError
obj_cols = df_vehiculos.select_dtypes(include=["object"]).columns
if len(obj_cols) > 0:
	# pandas 'string' dtype is arrow-friendly and avoids mixed-type issues (e.g. ints in object column)
	df_vehiculos[obj_cols] = df_vehiculos[obj_cols].astype("string")

# write parquet (avoid index if not needed)
df_vehiculos.to_parquet("veh_raw.parquet", index=False)

C:\Users\hiros\AppData\Local\Temp\ipykernel_30028\3582592511.py:18: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  parsed = pd.to_datetime(sample, errors="coerce", infer_datetime_format=True)
C:\Users\hiros\AppData\Local\Temp\ipykernel_30028\3582592511.py:18: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  parsed = pd.to_datetime(sample, errors="coerce", infer_datetime_format=True)
C:\Users\hiros\AppData\Local\Temp\ipykernel_30028\3582592511.py:18: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To

Converted to datetime: ['FECHA_INGRESO_RNT', 'año_archivo', 'FECHA_INGRESO_SERVICIO', 'FECHA _INGRESO_RNT', 'FECHA_INGRESO_RNTTT', 'FECHA_INGRESO', 'FECHA INGRESO RNT', 'FECHA_INGRESO_RNT.1', 'FECHA_INGRESO_RNTT']


ArrowTypeError: ("Expected bytes, got a 'int' object", 'Conversion failed for column MARCA with type object')

In [18]:
df_vehiculos.to_csv("vehiculos_extraidos.csv", index=False)


In [19]:
df_vehiculos

,REGION,FOLIO,TIPO_SERVICIO,PPU,FECHA_INGRESO_RNT,MARCA,MODELO,ANO_FABRICACION,CAPACIDAD,año_archivo,...,TIPO_SERVCIO,TIPO_COMBUSTIBLE,FECHA_INGRESO_RNT.1,TIPO_COMBUSTIBLE,COMBUSTIBLE,FECHA_INGRESO_RNTT,TIPO SERVICIO,LINEA,AÑO FABRICACION,AÑOFABRICACION
0,1.0,10,TAXI COLECTIVO RURAL,RW2882,2007-06-26,HYUNDAI,ACCENT GL 1.5,2007.0,4.0,2021-01-01,...,NaN,NaN,NaT,NaN,NaN,NaT,NaN,NaN,NaN,NaN
1,1.0,12,TAXI COLECTIVO RURAL,BCRH67,2019-10-07,HYUNDAI,ACCENT 1.6,2015.0,NaN,2021-01-01,...,NaN,NaN,NaT,NaN,NaN,NaT,NaN,NaN,NaN,NaN
2,1.0,18,TAXI COLECTIVO RURAL,DWCY37,2012-05-15,HYUNDAI,ACCENT RB GL 1.6,2012.0,4.0,2021-01-01,...,NaN,NaN,NaT,NaN,NaN,NaT,NaN,NaN,NaN,NaN
3,1.0,19,TAXI COLECTIVO RURAL,DGFX88,2011-08-30,HYUNDAI,ACCENT RB GL 1.6,2011.0,4.0,2021-01-01,...,NaN,NaN,NaT,NaN,NaN,NaT,NaN,NaN,NaN,NaN
4,1.0,24,TAXI COLECTIVO RURAL,FPCP93,2013-05-03,HYUNDAI,ACCENT RB GL 1.6,2013.0,4.0,2021-01-01,...,NaN,NaN,NaT,NaN,NaN,NaT,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10480787,16.0,500700,NaN,DKWJ44,NaT,SCANIA,K380B,NaN,44.0,2019-01-01,...,NaN,NaN,NaT,NaN,NaN,NaT,NaN,NaN,NaN,NaN
10480788,16.0,500936,NaN,BXDK27,NaT,MERCEDES BENZ,0500 RS,NaN,NaN,2019-01-01,...,NaN,NaN,NaT,NaN,NaN,NaT,NaN,NaN,NaN,NaN
10480789,16.0,500825,NaN,KXBT57,NaT,MERCEDES BENZ,OF 1724,NaN,46.0,2019-01-01,...,NaN,NaN,NaT,NaN,NaN,NaT,NaN,NaN,NaN,NaN
10480790,16.0,400080,NaN,WK7479,NaT,MERCEDES BENZ,LO 914 42,NaN,NaN,2019-01-01,...,NaN,NaN,NaT,NaN,NaN,NaT,NaN,NaN,NaN,NaN


In [ ]:
asdasd

In [ ]:
vehiculos_df = df_vehiculos.copy()

In [ ]:
df_vehiculos.columns

Index(['REGION', 'FOLIO', 'TIPO_SERVICIO', 'PPU', 'FECHA_INGRESO_RNT', 'MARCA',
       'MODELO', 'ANO_FABRICACION', 'CAPACIDAD', 'año_archivo', 'mes_archivo',
       'archivo', 'hoja', 'ANOFABRICACION', 'TIPOSERVICIO',
       'FECHA_INGRESO_SERVICIO', ' TIPO_SERVICIO', 'FECHA _INGRESO_RNT',
       'Tipo_Servicio', 'TIPO_SERVICCIO', 'FECHA_INGRESO_RNTTT',
       'FECHA_INGRESO', 'FECHA INGRESO RNT', 'TIPO_SERVCIO',
       'TIPO_COMBUSTIBLE', 'FECHA_INGRESO_RNT.1', 'TIPO_COMBUSTIBLE ',
       'COMBUSTIBLE', 'FECHA_INGRESO_RNTT', 'TIPO SERVICIO', 'LINEA',
       'AÑO FABRICACION', 'AÑOFABRICACION'],
      dtype='object')

In [ ]:

vehiculos_df = vehiculos_df.copy()
vehiculos_df.columns = [normalizar_texto(c) for c in vehiculos_df.columns]


In [ ]:
column_groups = {
    "tipo_servicio": [
        "tipo_servicio", "tiposervicio", "tipo_serviccio",
        "tipo_servcio", "tipo_servicio_", "tipo_servicio_1",
        "tipo_servicio1"
    ],
    "fecha_ingreso_rnt": [
        "fecha_ingreso_rnt", "fecha_ingreso_rntt", "fecha_ingreso_rnttt",
        "fecha_ingreso_rnt_1", "fecha_ingreso_rnt1",
        "fecha_ingreso_rnt_", "fecha_ingreso_rnt_"
    ],
    "combustible": [
        "combustible", "tipo_combustible", "tipo_combustible_",
        "tipo_combustible1"
    ],
    "fecha_ingreso": [
        "fecha_ingreso", "fecha_ingreso_servicio", "fecha_ingreso1"
    ],
    "ano_fabricacion": [
        "ano_fabricacion", "anofabricacion", "año_fabricacion",
        "ano_fabricacion_", "ano_fabricacion1"
    ]
}


In [ ]:
def unificar_columnas(df, groups):
    df = df.copy()
    for final_col, variantes in groups.items():
        existentes = [c for c in variantes if c in df.columns]
        if not existentes:
            continue
        
        df[final_col] = df[existentes].bfill(axis=1).iloc[:, 0]
        
        # eliminar columnas usadas excepto la final
        to_drop = [c for c in existentes if c != final_col]
        df = df.drop(columns=to_drop)
    return df


In [ ]:
vehiculos_df = unificar_columnas(vehiculos_df, column_groups)


MemoryError: 

In [ ]:
asdasd

### Consolidar columnas

In [ ]:
# Ordenar dataframe
column_order = [
    "folio",
    "region",
    "ppu",
    "linea",
    "marca",
    "modelo",
    "año_fabricacion",
    "capacidad",
    "tipo_servicio",
    "combustible",
    "fecha_ingreso",
    "fecha_ingreso_rnt",
    "mes",
    "ano"
]

column_order = [c for c in column_order if c in vehiculos_df.columns] + \
               [c for c in vehiculos_df.columns if c not in column_order]

vehiculos_df = vehiculos_df[column_order]

vehiculos_df.head()

,folio,region,ppu,linea,marca,modelo,año_fabricacion,capacidad,tipo_servicio,combustible,fecha_ingreso,fecha_ingreso_rnt,mes,ano,ano_fabricacion,anofabricacion
0,10,1.0,RW2882,nan,HYUNDAI,ACCENT GL 1.5,2007.0,4.0,TAXI COLECTIVO RURAL,nan,nan,2007-06-26,abril,2021,2007.0,NaN
1,12,1.0,BCRH67,nan,HYUNDAI,ACCENT 1.6,2015.0,NaN,TAXI COLECTIVO RURAL,nan,nan,2019-10-07,abril,2021,2015.0,NaN
2,18,1.0,DWCY37,nan,HYUNDAI,ACCENT RB GL 1.6,2012.0,4.0,TAXI COLECTIVO RURAL,nan,nan,2012-05-15,abril,2021,2012.0,NaN
3,19,1.0,DGFX88,nan,HYUNDAI,ACCENT RB GL 1.6,2011.0,4.0,TAXI COLECTIVO RURAL,nan,nan,2011-08-30,abril,2021,2011.0,NaN
4,24,1.0,FPCP93,nan,HYUNDAI,ACCENT RB GL 1.6,2013.0,4.0,TAXI COLECTIVO RURAL,nan,nan,2013-05-03,abril,2021,2013.0,NaN


In [ ]:
vehiculos_df

,folio,region,ppu,linea,marca,modelo,año_fabricacion,capacidad,tipo_servicio,combustible,fecha_ingreso,fecha_ingreso_rnt,mes,ano,ano_fabricacion,anofabricacion
0,10,1.0,RW2882,nan,HYUNDAI,ACCENT GL 1.5,2007.0,4.0,TAXI COLECTIVO RURAL,nan,nan,2007-06-26,abril,2021,2007.0,NaN
1,12,1.0,BCRH67,nan,HYUNDAI,ACCENT 1.6,2015.0,NaN,TAXI COLECTIVO RURAL,nan,nan,2019-10-07,abril,2021,2015.0,NaN
2,18,1.0,DWCY37,nan,HYUNDAI,ACCENT RB GL 1.6,2012.0,4.0,TAXI COLECTIVO RURAL,nan,nan,2012-05-15,abril,2021,2012.0,NaN
3,19,1.0,DGFX88,nan,HYUNDAI,ACCENT RB GL 1.6,2011.0,4.0,TAXI COLECTIVO RURAL,nan,nan,2011-08-30,abril,2021,2011.0,NaN
4,24,1.0,FPCP93,nan,HYUNDAI,ACCENT RB GL 1.6,2013.0,4.0,TAXI COLECTIVO RURAL,nan,nan,2013-05-03,abril,2021,2013.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10480787,500700,16.0,DKWJ44,nan,SCANIA,K380B,2012.0,44.0,nan,nan,nan,2013-06-03-15.58.03.000000,diciembre,2019,NaN,2012.0
10480788,500936,16.0,BXDK27,nan,MERCEDES BENZ,0500 RS,2009.0,NaN,nan,nan,nan,2015-11-10-17.26.01.000000,diciembre,2019,NaN,2009.0
10480789,500825,16.0,KXBT57,nan,MERCEDES BENZ,OF 1724,2019.0,46.0,nan,nan,nan,2018-11-09-00.00.00.000000,diciembre,2019,NaN,2019.0
10480790,400080,16.0,WK7479,nan,MERCEDES BENZ,LO 914 42,2007.0,NaN,nan,nan,nan,2007-08-10-12.00.00.000000,diciembre,2019,NaN,2007.0
